# 📘 Day 2: Long Short-Term Memory (LSTM) & Gated Recurrent Units (GRU)

**🎯 Goal:** Master LSTM and GRU architectures for advanced sequence modeling

**⏱️ Time:** 75-90 minutes

**🌟 Why This Matters for AI:**
- LSTMs powered pre-Transformer language models (GPT-1's predecessor!)
- Still used in production for time series, speech, and video
- Understanding gates → Understanding Transformers' attention mechanisms
- Critical for hybrid models combining RNNs with Transformers
- GRUs power many real-time AI systems (faster than LSTMs!)

---

## 🧠 The Problem: Vanishing Gradients in RNNs

### 📉 Recap from Day 1:

**Simple RNNs struggle with long-term dependencies:**

```
The cat, which had been wandering the streets for hours and was getting
very tired and hungry, finally _______ the food.

Answer: "ate"

Problem:
- "cat" is 15 words before "ate"
- Simple RNN gradient: 0.9^15 ≈ 0.2 (weak!)
- Can't learn the connection!
```

### ✅ The Solution: Gating Mechanisms!

**Key Insight:**
> Instead of forcing information through the same path (causing gradients to vanish),
> create **highways** that let important information flow unchanged!

**Think of it like:**
- **Simple RNN:** One-lane road with many toll booths (gradients shrink!)
- **LSTM/GRU:** Multi-lane highway with express lanes (information flows freely!)

---

## 🚪 LSTM: Long Short-Term Memory

**Invented in 1997 by Hochreiter & Schmidhuber**
- Revolutionized sequence modeling
- Powered Google Translate (pre-2017)
- Foundation of early speech recognition systems

### 🏗️ LSTM Architecture: The Cell State & Gates

**LSTM has TWO states instead of one:**
1. **Cell State (C_t):** Long-term memory highway
2. **Hidden State (h_t):** Short-term working memory

**Three Gates Control Information Flow:**

```
┌─────────────────────────────────────────┐
│           LSTM CELL                     │
│                                         │
│    C_{t-1} ──────────────────→ C_t     │  ← Cell State (Highway!)
│              ↓        ↓        ↓        │
│          [Forget] [Input] [Output]     │  ← Gates
│              ↓        ↓        ↓        │
│    h_{t-1} ───────────────────→ h_t    │  ← Hidden State
│      ↑                           ↑      │
│    x_t                          y_t     │
└─────────────────────────────────────────┘
```

### 🚪 The Three Gates:

#### 1️⃣ **Forget Gate (f_t)** - What to forget?
```
f_t = σ(W_f · [h_{t-1}, x_t] + b_f)

Decides: "Should I forget old information?"
Output: 0 (forget everything) to 1 (keep everything)

Example:
"The cat was hungry. The dog was tired."
       └─────────┘      ↑
      Forget cat     New subject!
```

#### 2️⃣ **Input Gate (i_t)** - What new info to add?
```
i_t = σ(W_i · [h_{t-1}, x_t] + b_i)           ← How much to update
C̃_t = tanh(W_C · [h_{t-1}, x_t] + b_C)       ← Candidate values

Decides: "What new information should I store?"

Example:
"The dog was tired"
       ↑
  Store "dog" as new subject!
```

#### 3️⃣ **Output Gate (o_t)** - What to output?
```
o_t = σ(W_o · [h_{t-1}, x_t] + b_o)
h_t = o_t * tanh(C_t)

Decides: "What parts of memory to expose?"
```

### 🔄 Complete LSTM Forward Pass:

```python
# Step 1: Forget Gate - decide what to forget from cell state
f_t = sigmoid(W_f @ [h_{t-1}, x_t] + b_f)

# Step 2: Input Gate - decide what new information to store
i_t = sigmoid(W_i @ [h_{t-1}, x_t] + b_i)
C̃_t = tanh(W_C @ [h_{t-1}, x_t] + b_C)

# Step 3: Update Cell State
C_t = f_t * C_{t-1} + i_t * C̃_t
      └──────────┘   └────────┘
      Forget old     Add new

# Step 4: Output Gate - decide what to output
o_t = sigmoid(W_o @ [h_{t-1}, x_t] + b_o)
h_t = o_t * tanh(C_t)
```

### 🎯 Why LSTM Solves Vanishing Gradients:

**Key: The Cell State Highway!**

```
Simple RNN Gradient Flow:
∂L/∂h_1 ← tanh' × W × tanh' × W × tanh' × W ← ∂L/∂h_4
           └─────────── Shrinks! ──────────┘

LSTM Cell State Gradient Flow:
∂L/∂C_1 ← f_2 × f_3 × f_4 ← ∂L/∂C_4
          └────────┘
       No repeated multiplication!
       Gradients flow through!
```

**Cell state acts like a conveyor belt** - information can flow unchanged for 100+ time steps!

---

## ⚡ GRU: Gated Recurrent Unit

**Invented in 2014 by Cho et al.**
- Simplified version of LSTM
- Fewer parameters → Faster training
- Often performs as well as LSTM!

### 🏗️ GRU Architecture: Simpler but Powerful

**Key Simplifications:**
1. Only ONE state (no separate cell state)
2. Only TWO gates (vs LSTM's three)
3. Fewer parameters → Faster!

```
┌─────────────────────────────────┐
│         GRU CELL                │
│                                 │
│   h_{t-1} ────────────→ h_t    │  ← Single state!
│        ↓      ↓         ↓       │
│    [Reset] [Update]             │  ← Only 2 gates!
│        ↓      ↓                 │
│      x_t    y_t                 │
└─────────────────────────────────┘
```

### 🚪 The Two Gates:

#### 1️⃣ **Update Gate (z_t)** - How much to update?
```
z_t = σ(W_z · [h_{t-1}, x_t] + b_z)

Acts like LSTM's forget + input gates combined!
- z_t = 0: Keep old state (ignore new input)
- z_t = 1: Replace with new state
```

#### 2️⃣ **Reset Gate (r_t)** - How much past to forget?
```
r_t = σ(W_r · [h_{t-1}, x_t] + b_r)

Controls how much past information to use when computing new state
```

### 🔄 Complete GRU Forward Pass:

```python
# Step 1: Reset Gate - decide what past info to use
r_t = sigmoid(W_r @ [h_{t-1}, x_t] + b_r)

# Step 2: Candidate state (using reset gate)
h̃_t = tanh(W_h @ [r_t * h_{t-1}, x_t] + b_h)

# Step 3: Update Gate - interpolate between old and new
z_t = sigmoid(W_z @ [h_{t-1}, x_t] + b_z)

# Step 4: Final state (weighted average)
h_t = (1 - z_t) * h_{t-1} + z_t * h̃_t
      └──────────────┘   └────────┘
      Keep old          Use new
```

---

## ⚖️ LSTM vs GRU: When to Use Each?

| Feature | LSTM | GRU |
|---------|------|-----|
| **Gates** | 3 (forget, input, output) | 2 (reset, update) |
| **States** | 2 (cell + hidden) | 1 (hidden only) |
| **Parameters** | More (~4x input size) | Fewer (~3x input size) |
| **Training Speed** | Slower | **Faster** ✅ |
| **Memory Usage** | Higher | **Lower** ✅ |
| **Performance** | Slightly better on complex tasks | **Similar for most tasks** ✅ |
| **Best For** | Very long sequences (>100 steps) | Most real-world tasks |

### 🎯 General Guidelines:

**Use LSTM when:**
- Very long sequences (100+ time steps)
- Complex dependencies
- Plenty of training data
- Computational resources available
- Examples: Long documents, music generation

**Use GRU when:**
- Medium-length sequences (10-100 steps)
- Need faster training
- Limited computational resources
- Real-time applications
- Examples: Chatbots, sentiment analysis, stock prediction

**Pro Tip:** Start with GRU! It's simpler and often works just as well.

---

## 🚀 Let's Build! Real AI Example 1: Sentiment Analysis with LSTM

**Project:** Movie review sentiment classifier
- Input: Text review ("This movie was amazing!")
- Output: Positive (😊) or Negative (😞)

**Why this matters:**
- Same architecture used in early ChatGPT models
- Foundation of understanding how LLMs process text
- Used in production for social media monitoring, customer feedback

Let's build it with TensorFlow!

In [ ]:
# Install required libraries
import sys
!{sys.executable} -m pip install tensorflow numpy matplotlib scikit-learn --quiet

print("✅ Libraries installed!")

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set random seed
np.random.seed(42)
tf.random.set_seed(42)

print("📚 Libraries loaded!")
print(f"TensorFlow version: {tf.__version__}")

### Step 1: Load IMDB Movie Review Dataset

**Dataset:**
- 50,000 movie reviews from IMDB
- 25,000 for training, 25,000 for testing
- Labels: 0 (negative) or 1 (positive)
- Pre-tokenized (words → integers)

In [ ]:
# Load IMDB dataset (top 10,000 most common words)
vocab_size = 10000
max_length = 200  # Truncate reviews to 200 words

print("📥 Loading IMDB dataset...")
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

print(f"\n✅ Dataset loaded!")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")
print(f"   Vocabulary size: {vocab_size}")
print(f"\n   Example review (first 10 words): {X_train[0][:10]}")
print(f"   Label: {y_train[0]} ({'Positive' if y_train[0] == 1 else 'Negative'})")

In [ ]:
# Decode a review to see actual text
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

print("📖 Example review (decoded):")
print(f"\nReview: {decode_review(X_train[0][:50])}...")
print(f"\nSentiment: {'Positive 😊' if y_train[0] == 1 else 'Negative 😞'}")

### Step 2: Preprocess Data

**Why pad sequences?**
- Reviews have different lengths
- Neural networks need fixed-size inputs
- Solution: Pad/truncate to max_length

In [ ]:
# Pad sequences to same length
X_train_pad = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

print("✅ Sequences padded!")
print(f"   X_train shape: {X_train_pad.shape}")
print(f"   X_test shape: {X_test_pad.shape}")
print(f"\n   Example padded sequence (last 10 values): {X_train_pad[0][-10:]}")
print("   (zeros = padding)")

### Step 3: Build LSTM Model

**Architecture:**
1. **Embedding Layer:** Convert word indices to dense vectors
2. **LSTM Layer:** Process sequence with long-term memory
3. **Dense Layer:** Classification (positive/negative)

**This is similar to how early GPT models worked!**

In [ ]:
# Build LSTM model
def build_lstm_model(vocab_size, embedding_dim=128, lstm_units=64):
    """
    Build LSTM sentiment classifier
    
    Architecture:
    - Embedding: Convert words to vectors
    - LSTM: Process sequence
    - Dense: Binary classification
    """
    model = keras.Sequential([
        # Embedding layer (learn word representations)
        layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        
        # LSTM layer (process sequence with memory)
        layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2),
        
        # Output layer (binary classification)
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

# Create model
lstm_model = build_lstm_model(vocab_size, embedding_dim=128, lstm_units=64)

print("🧠 LSTM Model created!")
print("\n📊 Model Architecture:")
lstm_model.summary()

In [ ]:
# Compile model
lstm_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✅ Model compiled!")
print("   Optimizer: Adam")
print("   Loss: Binary Cross-Entropy")
print("   Metrics: Accuracy")

### Step 4: Train LSTM Model

In [ ]:
# Train model
print("🚀 Training LSTM model...")
print("   This will take 5-10 minutes depending on hardware\n")

history_lstm = lstm_model.fit(
    X_train_pad, y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.2,
    verbose=1
)

print("\n✅ Training complete!")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
ax1.plot(history_lstm.history['accuracy'], label='Training', linewidth=2)
ax1.plot(history_lstm.history['val_accuracy'], label='Validation', linewidth=2)
ax1.set_title('LSTM Model Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss
ax2.plot(history_lstm.history['loss'], label='Training', linewidth=2)
ax2.plot(history_lstm.history['val_loss'], label='Validation', linewidth=2)
ax2.set_title('LSTM Model Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Step 5: Evaluate and Test

In [ ]:
# Evaluate on test set
print("🔬 Evaluating LSTM model...\n")
test_loss, test_accuracy = lstm_model.evaluate(X_test_pad, y_test, verbose=0)

print(f"✅ Test Results:")
print(f"   Test Loss: {test_loss:.4f}")
print(f"   Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"\n🎯 LSTM correctly classifies {test_accuracy*100:.1f}% of movie reviews!")

In [ ]:
# Test on sample reviews
sample_indices = [0, 10, 50, 100, 500]

print("🎬 Sample Predictions:\n")
for idx in sample_indices:
    review = decode_review(X_test[idx][:30])  # First 30 words
    true_sentiment = "Positive 😊" if y_test[idx] == 1 else "Negative 😞"
    
    # Predict
    pred = lstm_model.predict(X_test_pad[idx:idx+1], verbose=0)[0][0]
    pred_sentiment = "Positive 😊" if pred > 0.5 else "Negative 😞"
    confidence = pred if pred > 0.5 else 1 - pred
    
    print(f"Review: {review}...")
    print(f"True: {true_sentiment} | Predicted: {pred_sentiment} ({confidence*100:.1f}% confidence)")
    print("---")

## 🎨 Real AI Example 2: Character-Level Text Generation

**Project:** Train LSTM to generate text character by character

**How it works:**
- Input: Sequence of characters ("Hell")
- Output: Next character ("o")
- Generate: Sample next character, add to sequence, repeat!

**This is how early text generation worked!**
- Shakespeare text generation (Andrej Karpathy's famous demo)
- Music generation (character = note)
- Code generation (character = code token)

In [ ]:
# Sample text (you can replace with any text!)
text = """In the age of artificial intelligence, machine learning has transformed how we
interact with technology. From chatbots like ChatGPT to autonomous vehicles, AI is everywhere.
Deep learning, particularly transformers and neural networks, powers modern AI systems.
Understanding these foundations is crucial for building the next generation of AI applications."""

print("📚 Training text:")
print(text[:200] + "...")
print(f"\n   Text length: {len(text)} characters")

In [ ]:
# Create character mappings
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size_char = len(chars)

print(f"📊 Character vocabulary:")
print(f"   Unique characters: {vocab_size_char}")
print(f"   Characters: {chars}")

In [ ]:
# Create sequences for character prediction
seq_length = 40  # Use 40 characters to predict next one

sequences = []
next_chars = []

for i in range(len(text) - seq_length):
    sequences.append(text[i:i+seq_length])
    next_chars.append(text[i+seq_length])

print(f"🔢 Created {len(sequences)} sequences")
print(f"\nExample:")
print(f"   Input: '{sequences[0]}'")
print(f"   Target: '{next_chars[0]}'")

In [ ]:
# Prepare data for LSTM
X_char = np.zeros((len(sequences), seq_length, vocab_size_char), dtype=bool)
y_char = np.zeros((len(sequences), vocab_size_char), dtype=bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X_char[i, t, char_to_idx[char]] = 1
    y_char[i, char_to_idx[next_chars[i]]] = 1

print(f"✅ Data prepared!")
print(f"   X shape: {X_char.shape} (samples, time_steps, features)")
print(f"   y shape: {y_char.shape} (samples, classes)")

In [ ]:
# Build character-level LSTM
char_model = keras.Sequential([
    layers.LSTM(128, input_shape=(seq_length, vocab_size_char), return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(128),
    layers.Dropout(0.2),
    layers.Dense(vocab_size_char, activation='softmax')
])

char_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("🧠 Character-level LSTM Model:")
char_model.summary()

In [ ]:
# Train character model
print("🚀 Training character-level LSTM...\n")

history_char = char_model.fit(
    X_char, y_char,
    batch_size=128,
    epochs=30,
    validation_split=0.1,
    verbose=1
)

print("\n✅ Training complete!")

In [ ]:
# Generate text!
def generate_text(model, seed_text, length=200, temperature=0.5):
    """
    Generate text using trained LSTM
    
    Args:
        seed_text: Starting text (must be >= seq_length)
        length: Number of characters to generate
        temperature: Randomness (lower = more conservative)
    """
    generated = seed_text
    
    for _ in range(length):
        # Prepare input
        x_pred = np.zeros((1, seq_length, vocab_size_char))
        for t, char in enumerate(seed_text[-seq_length:]):
            x_pred[0, t, char_to_idx[char]] = 1
        
        # Predict next character
        predictions = model.predict(x_pred, verbose=0)[0]
        
        # Apply temperature
        predictions = np.log(predictions + 1e-7) / temperature
        predictions = np.exp(predictions) / np.sum(np.exp(predictions))
        
        # Sample next character
        next_idx = np.random.choice(len(predictions), p=predictions)
        next_char = idx_to_char[next_idx]
        
        generated += next_char
        seed_text += next_char
    
    return generated

# Generate text samples
seed = "In the age of artificial intelligence"

print("🎨 Generated Text Samples:\n")
print("=" * 60)

for temp in [0.2, 0.5, 1.0]:
    print(f"\nTemperature: {temp} {'(conservative)' if temp < 0.5 else '(creative)' if temp > 0.7 else '(balanced)'}")
    generated = generate_text(char_model, seed, length=150, temperature=temp)
    print(generated)
    print("-" * 60)

## 🎯 Why This Matters for Modern AI (2024-2025)

### 🤖 **From LSTMs to Transformers**

**The Evolution:**
```
2014: LSTM/GRU dominate NLP
  └── Seq2Seq models (translation)
  └── Early language models

2017: Attention mechanism added to LSTMs
  └── "Attention is All You Need" paper
  └── Birth of Transformers!

2018-2025: Transformers replace LSTMs
  ├── GPT (2018): 117M parameters
  ├── GPT-2 (2019): 1.5B parameters
  ├── GPT-3 (2020): 175B parameters
  ├── GPT-4 (2023): ~1.8T parameters (estimated)
  └── Claude 3.5 Sonnet (2024): Advanced reasoning
```

### 🔑 **LSTM Concepts That Survive in Transformers:**

1. **Gates → Attention**
   - LSTM gates decide what to remember/forget
   - Transformer attention decides what to focus on
   - Same principle: **selective information flow**

2. **Cell State → Residual Connections**
   - LSTM cell state: information highway
   - Transformer residuals: skip connections
   - Both solve vanishing gradients!

3. **Sequential Processing → Positional Encoding**
   - LSTMs: inherent order from recurrence
   - Transformers: explicit positional embeddings
   - Both preserve sequence information

### 📊 **Where LSTMs Still Shine (2024-2025):**

**1. Time Series Forecasting:**
- Stock prices, weather, energy demand
- More efficient than Transformers for numerical sequences
- Lower computational cost

**2. Speech Recognition:**
- Real-time audio processing
- Streaming applications (can't wait for full sequence)
- Used in Siri, Alexa, Google Assistant

**3. Video Analysis:**
- Temporal modeling across frames
- Action recognition
- Hybrid with CNNs (ConvLSTM)

**4. IoT & Edge Devices:**
- Smaller models (GRU especially)
- Lower memory footprint
- Real-time processing on mobile devices

**5. Hybrid Architectures:**
- Combining LSTMs with Transformers
- Best of both worlds!

### 🎯 **Key Takeaway:**
Understanding LSTMs/GRUs is essential because:
- They teach you how to handle sequences
- Gating mechanisms appear everywhere in modern AI
- Many production systems still use them
- They're the stepping stone to understanding Transformers!

---

## 🎯 Interactive Exercise 1: Build a GRU Model

**Challenge:** Modify the sentiment analysis model to use GRU instead of LSTM!

**Your Task:**
1. Create a GRU model with similar architecture
2. Train for 5 epochs
3. Compare performance with LSTM
4. Compare training time

In [ ]:
# YOUR CODE HERE!

# TODO: Build GRU model (replace LSTM with GRU layer)
# TODO: Compile with same settings
# TODO: Train and time the training
# TODO: Evaluate and compare with LSTM

print("Complete the exercise above!")

### ✅ Solution (Try on your own first!)

In [ ]:
# SOLUTION
import time

# Build GRU model
gru_model = keras.Sequential([
    layers.Embedding(vocab_size, 128, input_length=max_length),
    layers.GRU(64, dropout=0.2, recurrent_dropout=0.2),  # GRU instead of LSTM!
    layers.Dense(1, activation='sigmoid')
])

gru_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("🧠 GRU Model:")
gru_model.summary()

# Train and time
print("\n🚀 Training GRU model...")
start_time = time.time()

history_gru = gru_model.fit(
    X_train_pad, y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.2,
    verbose=1
)

gru_train_time = time.time() - start_time

# Evaluate
gru_test_loss, gru_test_acc = gru_model.evaluate(X_test_pad, y_test, verbose=0)

# Compare
print("\n📊 LSTM vs GRU Comparison:")
print("=" * 50)
print(f"{'Metric':<25} {'LSTM':<15} {'GRU':<15}")
print("=" * 50)
print(f"{'Test Accuracy':<25} {test_accuracy:.4f}          {gru_test_acc:.4f}")
print(f"{'Parameters':<25} {lstm_model.count_params():<15} {gru_model.count_params():<15}")
print(f"{'Training Time':<25} {gru_train_time:.2f}s")
print("=" * 50)

if gru_test_acc > test_accuracy:
    print("\n✅ GRU slightly outperformed LSTM!")
elif abs(gru_test_acc - test_accuracy) < 0.01:
    print("\n✅ GRU and LSTM performed similarly!")
    print("   💡 GRU is often preferred: fewer parameters, faster training")
else:
    print("\n✅ LSTM slightly outperformed GRU!")
    
print(f"\n🎯 Parameter reduction: {(1 - gru_model.count_params()/lstm_model.count_params())*100:.1f}%")

## 🎯 Interactive Exercise 2: Bidirectional LSTM

**Challenge:** Create a Bidirectional LSTM!

**What is Bidirectional RNN?**
- Processes sequence both forward AND backward
- Sees future context too!
- Better for understanding full context

**Example:**
```
"The movie was ____"
Forward LSTM: Only sees "The movie was"
Backward LSTM: Sees what comes after!

Useful when you have full sequence (not streaming)
```

In [ ]:
# YOUR CODE HERE!

# TODO: Build model with Bidirectional wrapper
# Hint: layers.Bidirectional(layers.LSTM(64))

print("Complete the exercise above!")

### ✅ Solution

In [ ]:
# SOLUTION

# Build Bidirectional LSTM
bilstm_model = keras.Sequential([
    layers.Embedding(vocab_size, 128, input_length=max_length),
    layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    layers.Dense(1, activation='sigmoid')
])

bilstm_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("🧠 Bidirectional LSTM Model:")
bilstm_model.summary()

# Train
print("\n🚀 Training Bidirectional LSTM...")
history_bilstm = bilstm_model.fit(
    X_train_pad, y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.2,
    verbose=1
)

# Evaluate
bilstm_test_loss, bilstm_test_acc = bilstm_model.evaluate(X_test_pad, y_test, verbose=0)

print(f"\n✅ Bidirectional LSTM Test Accuracy: {bilstm_test_acc:.4f}")
print(f"\n📊 Comparison:")
print(f"   Regular LSTM: {test_accuracy:.4f}")
print(f"   Bidirectional: {bilstm_test_acc:.4f}")
print(f"\n💡 Bidirectional sees both past and future context!")

## 🎉 Congratulations!

**You just learned:**
- ✅ LSTM architecture with 3 gates (forget, input, output)
- ✅ GRU architecture (simplified LSTM with 2 gates)
- ✅ How gates solve the vanishing gradient problem
- ✅ When to use LSTM vs GRU
- ✅ Built sentiment analysis with LSTM (like early ChatGPT!)
- ✅ Character-level text generation (foundation of GPT)
- ✅ Bidirectional RNNs for full context understanding
- ✅ How LSTM concepts evolved into Transformers

### 🎯 Key Takeaways:

1. **Gates enable long-term memory**
   - Forget gate: Remove irrelevant information
   - Input gate: Add new information
   - Output gate: Control what to expose
   - Cell state: Information highway!

2. **GRU is often better in practice**
   - Fewer parameters (25% fewer than LSTM)
   - Faster training
   - Similar performance
   - Use GRU unless you need very long sequences

3. **LSTM/GRU → Transformers evolution**
   - Gates → Attention mechanisms
   - Cell state → Residual connections
   - Sequential processing → Parallel processing
   - Understanding LSTMs helps you understand GPT!

4. **Still relevant in 2024-2025**
   - Time series forecasting
   - Speech recognition (streaming)
   - Video analysis
   - Edge devices (smaller models)

---

**📚 Next Lesson:** Day 3 - Sequence-to-Sequence Models
- Encoder-Decoder architecture
- Attention mechanisms (foundation of Transformers!)
- Machine translation
- Building a chatbot
- Name generation with RNNs

---

**💬 Remember:**

*"LSTMs and GRUs were the breakthrough that made modern NLP possible. While Transformers have replaced them for most language tasks, understanding these architectures is essential for grasping how attention mechanisms work. The gating concepts you learned today appear everywhere in modern AI - from GPT's attention to diffusion models' conditioning!"* 🚀

---

**🔗 Connections to Modern AI:**
- **Pre-Transformer Era:** LSTMs dominated NLP (2014-2017)
- **Attention Mechanisms:** Born from LSTM limitations
- **Transformers:** Replaced sequential processing with parallel attention
- **Hybrid Models:** Combining LSTMs with Transformers for best results
- **Production Systems:** Many still use GRUs for efficiency